In [16]:
import collections
import pandas as pd
import numpy as np
from collections import Counter

import pymorphy2 as pm
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC



# Подгатавливаем тексты и теги
Читаем из файла данные, избавляемся от ненужных

In [2]:
df = pd.read_csv('D:\Max\docs_cluster\lenta\lenta_data.csv')
data = list(zip(df['text'].tolist(), df['tags'].tolist()))
tags = sorted(set(df['tags'].tolist()))

tags_cnt = list(Counter([d[1] for d in data]).items())
tags_accepted = [tag[0] for tag in tags_cnt if tag[0] != 'Все' and tag[1] > 100]
print(tags_cnt)
print(tags_accepted)

texts = [d[0] for d in data if d[1] in tags_accepted]
text_tags = [d[1] for d in data if d[1] in tags_accepted]

[('Легпром', 4), ('События', 2234), ('Мир', 1274), ('Стиль', 1096), ('Ресурсы', 690), ('Россия', 376), ('Оружие', 4120), ('ТВ и радио', 1951), ('Пресса', 1424), ('Книги', 939), ('Первая мировая', 65), ('Внешний вид', 211), ('Хоккей', 381), ('Мемы', 336), ('Все', 450709), ('Следствие и суд', 4165), ('Театр', 882), ('Криминал', 1497), ('Архитектура', 232), ('Движение', 423), ('Молдавия', 333), ('Теннис', 930), ('Явления', 1345), ('Средняя Азия', 936), ('Производители', 36), ('Гаджеты', 1816), ('Инновации', 1), ('Наследие', 14), ('Искусство', 1495), ('Еда', 332), ('Мнения', 359), ('Часы', 355), ('Москва', 1175), ('Аналитика рынка', 28), ('Вещи', 305), ('Туризм', 30), ('Банки', 1876), ('Бокс и ММА', 1379), ('Деньги', 987), ('Кино', 6592), ('История', 203), ('Происшествия', 10858), ('Госрегулирование', 41), ('Coцсети', 2039), ('Культпросвет', 1), ('Госэкономика', 9190), ('Техника', 936), ('Игры', 1954), ('Полиция и спецслужбы', 905), ('Выборы', 4), ('Технологии', 200), ('Вооруженные силы', 

# Класс TextClassifier
Сделал, чтобы можно было один раз загнать корпус и после этого играться с параметрами у классифкаторов. Пока получилось не очень, если честно...

In [22]:
class TextClassifier:
    
    
    def __init__(self, texts, text_tags, selections_num):
        le = LabelEncoder()
        
        self.texts = texts
        le.fit(list(set(text_tags)))
        self.text_tags = le.transform(text_tags)
        
        self.make_selections(selections_num)
        self.__lemmatize()
        
        
    def __lemmatize(self):
        print("Making vocab")
        morph = pm.MorphAnalyzer()
        tokenizer = TweetTokenizer()
        tokens = []
        n = len(self.texts)
        for i, text in enumerate(self.texts):
            self.__check_percentage(i, n)
                
            tokens.append([t.lower() for t in tokenizer.tokenize(text) if t.isalnum()])
        print('...')

        words_set = set().union(*tokens)
        print('Vocab length = ' + str(len(words_set)))
        vocab = {t: morph.parse(t)[0].normal_form for t in words_set}
        print('Finished vocab, lemmatizing')
        lemmatized_texts = []
        for toks in tokens:
            lemmatized_texts.append(' '.join([vocab.get(t) for t in toks if vocab.get(t) is not None]))
            
        self.texts = lemmatized_texts
        print("Done")
            
        
    def make_selections(self, selections_num):
        self.selections_num = selections_num
        self.corpus_size = len(self.texts)
        self.selection_size = self.corpus_size // self.selections_num
        
    
    def preset(self, norm='l2', use_idf=True, max_df=100000, min_df=10, stop_words=True):
        self.tfidfTransformer = TfidfTransformer(use_idf=use_idf)
        if stop_words:
            stopwords_ru = [w for w in stopwords.words("russian")]
        else:
            stopwords_ru = None
            
        self.vectorizer = CountVectorizer(max_df=max_df, min_df=min_df, stop_words=stopwords_ru)
        
        
    def __train_n_test(self, clf, clf_name):
        print("Training " + clf_name)
        models = []
        
        #Training
        for i in range(self.selections_num - 1):
            print("Selection #" + str(i))
            start = i * self.selection_size
            end = start + self.selection_size
            
            models.append(clf.fit(self.texts[start:end], self.text_tags[start:end]))
        
        #Testing
        start = (self.selections_num - 1) * self.selection_size
        test_selection_cnt = self.corpus_size - (self.selections_num - 1) * self.selection_size
        test_tags = self.text_tags[start:]

        model_predictions = [model.predict(texts[start:]) for model in models]
        prediction = []
        for i in range(len(model_predictions[0])):
            predictions = [p[i] for p in model_predictions]
            prediction.append(max(set(predictions), key=predictions.count))


        prediction = np.array(prediction)
        log_reg_accuracy = accuracy_score(test_tags, prediction)
        print("Method accuracy = " + str(log_reg_accuracy))
        return models
       
    
    
    #Classifiers
    def log_reg(self, C=None):
        clf = Pipeline([('vect', self.vectorizer), 
             ('tfidf', self.tfidfTransformer),
             ('clf', LogisticRegression(C=C))
               ])
        
        return self.__train_n_test(clf, "logistic regression")
    
    
    def random_forest(self, max_depth=None, random_state=None):
        clf = Pipeline([('vect', self.vectorizer), 
             ('tfidf', self.tfidfTransformer),
             ('clf', RandomForestClassifier(max_depth=max_depth, random_state=random_state))
               ])
        
        return self.__train_n_test(clf, "random forest")
    
    
    def decision_tree(self, max_depth=None, random_state=None):
        clf = Pipeline([('vect', self.vectorizer), 
             ('tfidf', self.tfidfTransformer),
             ('clf', DecisionTreeClassifier(max_depth=max_depth, random_state=random_state))
               ])
        
        return self.__train_n_test(clf, "decision tree")
    
    
    def radom_gradient(self):
        clf = Pipeline([('vect', self.vectorizer), 
             ('tfidf', self.tfidfTransformer),
             ('clf', SGDClassifier())
               ])
        
        return self.__train_n_test(clf, "random gradient")
    
    
    def adaboost(self, random_state=None):
        clf = Pipeline([('vect', self.vectorizer), 
             ('tfidf', self.tfidfTransformer),
             ('clf', AdaBoostClassifier(random_state=random_state))
               ])
        
        return self.__train_n_test(clf, "AdaBoostClassifier")
    
    
    def gaussian(self):
        clf = Pipeline([('vect', self.vectorizer), 
             ('tfidf', self.tfidfTransformer),
             ('clf', GaussianNB())
               ])

        return self.__train_n_test(clf, "gauss")
    
    
    def linearSVC(self, C=None):
        clf = Pipeline([('vect', self.vectorizer), 
             ('tfidf', self.tfidfTransformer),
             ('clf', LinearSVC(C=C))
               ])

        return self.__train_n_test(clf, "linear SVC")
    
      
        
    def __check_percentage(self, i, n):
        r = i / n * 100
        if r > 10 and  r < 10.0005:
            print(str(r) + '%')
        elif r > 30 and  r < 30.0005:
            print(str(r) + '%')
        elif r > 50 and  r < 50.0005:
            print(str(r) + '%')
        elif r > 70 and  r < 70.0005:
            print(str(r) + '%')
        elif r > 90 and  r < 90.0005:
            print(str(r) + '%')
    
    
    
'''
Вместо того, чтобы писать метод для каждого классификатора, лучше сделать как-то так (позже переделаю)
classifiers = [
    ("SGD", SGDClassifier()),
    ("ASGD", SGDClassifier(average=True)),
    ("Perceptron", Perceptron()),
    ("Passive-Aggressive I", PassiveAggressiveClassifier(loss='hinge',
                                                         C=1.0)),
    ("Passive-Aggressive II", PassiveAggressiveClassifier(loss='squared_hinge',
                                                          C=1.0)),
    ("SAG", LogisticRegression(solver='sag', tol=1e-1, C=1.e4 / X.shape[0]))
]
'''

In [15]:
text_classifier = TextClassifier(texts, text_tags, 6)
text_classifier.preset()

text_classifier.log_reg(C=10)    #лучший результат при C = 10 (~54%)
text_classifier.random_forest()   #37% без парамеров
text_classifier.radom_gradient()  #быстрее, чем Logistic regression, точность 58%
text_classifier.decision_tree() 
text_classifier.adaboost() 
text_classifier.gaussian()       #пока не смотрел
text_classifier.linearSVC(C=10)  #пока не смотрел

print('_')

Selection #0
Training random forest
Selection #1
Training random forest
Selection #2
Training random forest
Selection #3
Training random forest
Selection #4
Training random forest
Method accuracy = 0.363979193758
_


__Логистическая регрессия__ лучше всего при c = 10, точность *54%*

__Random forest__ без параметров *37%*

__SGDC__ быстрый, точность *58%*

__Решающее дерево__ без параметров *24%*

__Adaboost__ *14%*

# Ниже черновик
# ы
# ы

In [ ]:
import pymorphy2 as pm
from nltk.tokenize import TweetTokenizer

morph = pm.MorphAnalyzer()
tokenizer = TweetTokenizer()
tokens = []
vocab = []
for text in texts:
    tokens.extend([t.lower() for t in tokenizer.tokenize(text) if t.isalnum()])

tokens = set(tokens)
vocab = {t: morph.parse(t)[0].normal_form for t in tokens}

In [4]:
corpus_size = len(texts)
selections_num = 6      # На сколько частей разбиваем датасет
selection_size = corpus_size // selections_num
print(selection_size)
print(corpus_size - (selections_num - 1) * selection_size)


for i in range(selections_num - 1):
    start = i * selection_size
    end = start + selection_size
    print(str(start) + ' - ' + str(end))

30759
30760
0 - 30759
30759 - 61518
61518 - 92277
92277 - 123036
123036 - 153795


Тренируемся на первых пяти датасетах без лемматизации, проверяемся на шестом

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk.corpus import stopword


def check_percentage(i, n):
    r = i / n * 100
    if r > 10 and  r < 10.01:
        print(r)
    elif r > 30 and  r < 30.01:
        print(r)
    elif r > 50 and  r < 50.01:
        print(r)
    elif r > 70 and  r < 70.01:
        print(r)
    elif r > 90 and  r < 90.01:
        print(r)
    
stopwords_ru = [w for w in stopwords.words("russian")]

lin_reg_clf = Pipeline([('vect', CountVectorizer(stop_words = stopwords_ru)), 
             ('tfidf', TfidfTransformer()),
             ('clf', LogisticRegression())
               ])
                
lin_reg_models = []

for i in range(selections_num - 1):
    print("Selection #" + str(i))
    start = i * selection_size
    end = start + selection_size
    print("Training logistic regression")
    curr_texts = texts[start:end]
    normalized_texts = []
    
    
    n = len(curr_texts)
    for i, text in enumerate(curr_texts):
        check_percentage(i, n)
        tokens = tokenizer.tokenize(text)
        tokens = [morph.parse(t.lower())[0].normal_form for t in tokens if t.isalnum()]
        normalized_texts.append(' '.join(tokens))
        if i % 1000 == 0:
            print("Yay " + str(i))
    print("Done lemmatization")
    
    
    lin_reg_models.append(lin_reg_clf.fit(normalized_texts, text_tags[start:end]))

Selection #0
Training logistic regression
Yay 0
Yay 1000
Yay 2000
Yay 3000
10.000325108098442
10.003576189082871
10.006827270067298
Yay 4000
Yay 5000
Yay 6000
Yay 7000
Yay 8000
Yay 9000
30.000975324295325
30.004226405279756
30.007477486264182
Yay 10000
Yay 11000
Yay 12000
Yay 13000
Yay 14000
Yay 15000
50.00162554049221
50.00487662147665
50.00812770246107
Yay 16000
Yay 17000
Yay 18000
Yay 19000
Yay 20000
Yay 21000
70.0022757566891
70.00552683767353
70.00877791865796
Yay 22000
Yay 23000
Yay 24000
Yay 25000
Yay 26000
Yay 27000
90.00292597288599
90.0061770538704
90.00942813485484
Yay 28000
Yay 29000
Yay 30000
Done lemmatization
Selection #1
Training logistic regression
Yay 0
Yay 1000
Yay 2000
Yay 3000
10.000325108098442
10.003576189082871
10.006827270067298
Yay 4000
Yay 5000
Yay 6000
Yay 7000
Yay 8000
Yay 9000
30.000975324295325
30.004226405279756
30.007477486264182
Yay 10000
Yay 11000
Yay 12000
Yay 13000
Yay 14000
Yay 15000
50.00162554049221
50.00487662147665
50.00812770246107
Yay 16000
Y

In [6]:
from sklearn.metrics import accuracy_score


start = (selections_num - 1) * selection_size
test_selection_cnt = corpus_size - (selections_num - 1) * selection_size
test_tags = text_tags[start:]

'''
X_test_counts = count_vect.fit(texts[start:])
X_test_tfidf = tfidf_transformer.fit(X_test_counts)
'''

lin_reg_predictions = [model.predict(texts[start:]) for model in lin_reg_models]
prediction = []
#multinom_predictions = [model.predict(X_test_tfidf) for model in multinom_models
for i in range(len(lin_reg_predictions[0])):
    predictions = [p[i] for p in lin_reg_predictions]
    prediction.append(max(set(predictions), key=predictions.count))


prediction = np.array(prediction)
log_reg_accuracy = accuracy_score(test_tags, prediction)
print(log_reg_accuracy)

0.517067620286


In [44]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, twenty_train.target)

In [7]:
import pandas as pd
df = pd.read_csv('D:\Max\docs_cluster\lenta\lenta_data.csv')

In [13]:
df[:3]

,tags,text,title,topic,url
0,Деловой климат,Заместитель председателя правительства Аркадий...,Правительство прокомментировало идею о запрете...,Бизнес,https://lenta.ru/news/2017/04/01/24hours/
1,События,Монреальская конвенция об унификации правил во...,Эксперт отвел год на окончательную ратификацию...,Путешествия,https://lenta.ru/news/2017/03/31/motrealwork/
2,Общество,Сотни байкеров в столице Аргентины Буэнос-Айре...,Аргентинские байкеры устроили акцию протеста п...,Мир,https://lenta.ru/news/2017/03/30/bikers/


In [3]:
# ['tags', 'text', 'title', 'topic', 'url']
tags = [' спецпроект',
 'Coцсети',
 'Авто',
 'Автобизнес',
 'Аналитика рынка',
 'Архитектура',
 'Банки',
 'Баскетбол',
 'Безопасность ',
 'Белоруссия',
 'Бокс и ММА',
 'Вещи',
 'Вирусные ролики',
 'Вкусы',
 'Внешний вид',
 'Вооружение',
 'Вооруженные силы',
 'Все',
 'Выборы',
 'Гаджеты',
 'Госрегулирование',
 'Госэкономика',
 'Движение',
 'Деловой климат',
 'Деньги',
 'Достижения',
 'Еда',
 'Звери',
 'Зимние виды',
 'Игры',
 'Инновации',
 'Инструменты',
 'Интернет',
 'Искусство',
 'История',
 'Кавказ',
 'Казахстан',
 'Катастрофы',
 'Кино',
 'Книги',
 'Компании',
 'Конфликты',
 'Космос',
 'Криминал',
 'Крым',
 'Культпросвет',
 'Легпром',
 'Летние виды',
 'Люди',
 'Мемы',
 'Мир',
 'Мировой бизнес',
 'Мировой опыт',
 'Мнения',
 'Молдавия',
 'Москва',
 'Музыка',
 'Наследие',
 'Наука',
 'Общество',
 'Оружие',
 'Первая мировая',
 'Политика',
 'Полиция и спецслужбы',
 'Пресса',
 'Преступность',
 'Прибалтика',
 'Производители',
 'Происшествия',
 'Регионы',
 'Реклама',
 'Ресурсы',
 'Россия',
 'Рынки',
 'Следствие и суд',
 'События',
 'Средняя Азия',
 'Стиль',
 'Страноведение',
 'ТВ и радио',
 'Театр',
 'Теннис',
 'Техника',
 'Технологии',
 'Туризм',
 'Украина',
 'Финансы компаний',
 'Футбол',
 'Хоккей',
 'Часы',
 'Экология',
 'Явления']

num_tags = [{'tag': tag, 'id': i} for i, tag in enumerate(tags)]

texts = []
text_tags = []

with codecs.open('D:\Max\docs_cluster\lenta\lenta_data.csv', 'r', 'utf_8_sig') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        if i == 0:
            continue
        
        texts.append(row[1])
        text_tags.append(next((tag['id'] for tag in num_tags if tag['tag'] == row[0]), None))